In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Load the data and preprocess it
df = pd.read_csv('/content/drive/MyDrive/Thesis2024/fer2013.csv')
df = df[(df['emotion'] != 1) & (df['emotion'] != 5)]  # Remove disgust and surprise emotions

pixels = df['pixels'].tolist()
X = np.array([np.fromstring(pixel, dtype=int, sep=' ') for pixel in pixels])
X = X.reshape(-1, 48, 48, 1) / 255.0  # Normalize pixel values
y = df['emotion'].values

encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = pd.get_dummies(y).values  # Convert to one-hot encoding

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))  # Additional dense layer
model.add(Dense(5, activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy on test set: {accuracy * 100:.2f}%')

# Save the model
model.save('/content/drive/MyDrive/Thesis2024/enhanced_face_emotion_detector_model.h5')

Epoch 1/50
196/196 [==============================] - 5s 15ms/step - loss: 1.5835 - accuracy: 0.2854 - val_loss: 1.5681 - val_accuracy: 0.2907
Epoch 2/50
196/196 [==============================] - 2s 11ms/step - loss: 1.5351 - accuracy: 0.3306 - val_loss: 1.4756 - val_accuracy: 0.3711
Epoch 3/50
196/196 [==============================] - 2s 11ms/step - loss: 1.4440 - accuracy: 0.3957 - val_loss: 1.4232 - val_accuracy: 0.4198
Epoch 4/50
196/196 [==============================] - 2s 11ms/step - loss: 1.3767 - accuracy: 0.4318 - val_loss: 1.3444 - val_accuracy: 0.4569
Epoch 5/50
196/196 [==============================] - 2s 12ms/step - loss: 1.3298 - accuracy: 0.4540 - val_loss: 1.2933 - val_accuracy: 0.4708
Epoch 6/50
196/196 [==============================] - 3s 13ms/step - loss: 1.2866 - accuracy: 0.4713 - val_loss: 1.2703 - val_accuracy: 0.4826
Epoch 7/50
196/196 [==============================] - 3s 13ms/step - loss: 1.2565 - accuracy: 0.4877 - val_loss: 1.2384 - val_accuracy: 0.4952

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
